In [1]:
import torch
import torch.nn as nn

In [2]:

class Model4_6(nn.Module):
    def __init__(self):
        super(Model4_6, self).__init__() 
        self.cnn = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.maxpool=nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32*14*14, 10)
        
    def forward(self, x):
        out = self.cnn(x)
        out = self.relu(out)
        out = self.maxpool(out)
        out=out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

model4_6 = Model4_6() 

In [3]:
class Model4_7(nn.Module):
    def __init__(self):
        super(Model4_7, self).__init__() 
        self.layer1 = nn.Sequential(nn.Conv2d(1,16, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2))        
        self.fc1 = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out= self.layer1(x)
        out= self.layer2(out)
        out=out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

model4_7 = Model4_7() 

In [4]:
import torchvision
import torchvision.transforms as transforms
batchSize = 100
trainSet = torchvision.datasets.MNIST(root='./data', train = True, transform=transforms.ToTensor(), download=True)
trainLoader = torch.utils.data.DataLoader(dataset=trainSet, batch_size=batchSize, shuffle = True)
testSet = torchvision.datasets.MNIST(root='./data', train = False, transform=transforms.ToTensor(), download=True)
testLoader = torch.utils.data.DataLoader(dataset=testSet, batch_size=batchSize, shuffle = True)

In [5]:
import torch.optim as optim
import time

def benchmark(trainLoader, model, epochs=1, lr=0.01):
    model.__init__()
    start=time.time()
    optimiser = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(trainLoader):
            optimiser.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()
    print('Accuracy: {0:.4f}'.format(accuracy(testLoader,model)))
    print('Training time: {0:.2f}'.format(time.time() - start))       

In [6]:
def accuracy(testLoader,model):    
    correct, total = 0, 0
    with torch.no_grad():
        for data in testLoader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()      
    return(correct / total)

In [7]:
print('1 convolutional layers')
benchmark(trainLoader,model4_6, lr=0.1)
print('2 convolutional layers')
benchmark(trainLoader,model4_7, lr=0.1)

1 convolutional layers
Accuracy: 0.9654
Training time: 50.90
2 convolutional layers
Accuracy: 0.9743
Training time: 84.47


In [30]:
parameters=list((model.parameters()))
for parameter in parameters:
    print(parameter.size())

torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


1568